In [8]:
# 1. Extract entities from raw SR texts (PDF → TXT first if needed)
!python -m src.SR.data_generation.ner_extractor --corpus infectious_disease_papers --out dataset_scaffold/infection_ner_llm.jsonl --llm --model llama3.1:8b --verbose

2025-12-31 03:59:11,643 - DEBUG - Verbose logging enabled
2025-12-31 03:59:11,643 - INFO - ======================================================================
2025-12-31 03:59:11,643 - INFO - NER EXTRACTION PIPELINE - 55 ENTITY TYPES
2025-12-31 03:59:11,643 - INFO - ======================================================================
2025-12-31 03:59:11,643 - INFO - Corpus: /Users/awotoroebenezer/Desktop/SenseBackend/infectious_disease_papers
2025-12-31 03:59:11,643 - INFO - Output: /Users/awotoroebenezer/Desktop/SenseBackend/dataset_scaffold/infection_ner_llm.jsonl
2025-12-31 03:59:11,643 - INFO - LLM: Enabled
2025-12-31 03:59:11,643 - INFO - Workers: 1
2025-12-31 03:59:11,643 - INFO - Resume: False
2025-12-31 03:59:11,643 - INFO - Total entity types: 55
2025-12-31 03:59:11,643 - INFO - ======================================================================
2025-12-31 03:59:11,643 - INFO - Found 6 text files
2025-12-31 03:59:11,643 - INFO - Files to process: 6
2025-12-31 03:59:11,

In [11]:
!python -m src.SR.data_generation.qa_generator \
  --ner ./dataset_scaffold/infection_ner_llm.jsonl \
  --output qa_dataset \
  --train-split 0.7 --val-split 0.15 --test-split 0.15 \
  --seed 42

2025-12-31 04:04:48,361 - INFO - ============================================================
2025-12-31 04:04:48,361 - INFO - QA DATASET GENERATION
2025-12-31 04:04:48,361 - INFO - ============================================================
2025-12-31 04:04:48,361 - INFO - Input NER file: /Users/awotoroebenezer/Desktop/SenseBackend/dataset_scaffold/infection_ner_llm.jsonl
2025-12-31 04:04:48,361 - INFO - Output directory: /Users/awotoroebenezer/Desktop/SenseBackend/qa_dataset
2025-12-31 04:04:48,361 - INFO - Train/Val/Test split: 0.70/0.15/0.15
2025-12-31 04:04:48,361 - INFO - Random seed: 42
2025-12-31 04:04:48,361 - INFO - ============================================================
2025-12-31 04:04:48,361 - INFO - Loaded 12 NER results from dataset_scaffold/infection_ner_llm.jsonl
2025-12-31 04:04:48,362 - INFO - Generated 20 QA pairs for hiv_hpv_review
2025-12-31 04:04:48,362 - INFO - Generated 15 QA pairs for malaria_modeling_review
2025-12-31 04:04:48,362 - INFO - Generated 11 

In [12]:
!head -3 dataset_scaffold/infection_ner_llm.jsonl | sed 's/\\n/ /g' | cut -c1-300


{"doc_id": "hiv_hpv_review", "text": "A systematic review of HPV vaccination in HIV-positive populations.  We conducted a systematic review and meta-analysis of 25 studies from PubMed, MEDLINE, and Embase. The literature search was conducted up to December 2023. Studies were included if they reporte
{"doc_id": "malaria_modeling_review", "text": "Scoping review of mathematical modeling for malaria vaccine deployment.  This scoping review included 42 modeling studies from 2010-2022. Models included SEIR compartmental models (n=28) and agent-based models (n=14).  Studies were conducted in high-bur
{"doc_id": "paper1", "text": "A systematic review of antiretroviral therapy adherence in Sub-Saharan Africa. 42 studies with 15,000 HIV-positive patients were included. Adherence to ART was 78% (95% CI 72-84%). Barriers included stigma and distance to clinics. Methods: We searched PubMed, Scopus, an


In [13]:
!head -3 qa_dataset/train/train.jsonl | jq .
!cat qa_dataset/dataset_statistics.json | jq .


{
  "id": "tb_economic_review_q66_SEARCH_TERMS",
  "doc_id": "tb_economic_review",
  "question": "What keywords or MeSH terms were used in the search?",
  "context": "Rapid review of economic evaluations for tuberculosis vaccines.\n\nWe performed a rapid review of 18 economic evaluations published between 2015-2023.\nDatabase searches included PubMed, Embase, and Web of Science using terms: \"tuberculosis vaccine\" AND (\"cost\" OR \"economic\").\n\nStudies were from low- and middle-income countries (LMICs) in WHO regions: AFRO, SEARO, and EMRO.\nVaccine types included: M72/AS01E (efficacy 54%), VPM1002, and BCG revaccination.\n\nResults: Incremental cost-effectiveness ratios ranged from $120 to $2,500 per DALY averted.\nVaccination was cost-saving in 8 studies and cost-effective in 10 studies (ICER < GDP per capita).\n\nFactors influencing cost-effectiveness: vaccine price ($1.50-$8.00 per dose), delivery strategy (school-based vs routine), and coverage (65%-90%).\nSensitivity analysi

In [23]:
!python -m src.SR.BI.BiEncoderPipeline \
    --qa-train qa_dataset/train/train.jsonl \
    --qa-val qa_dataset/validation/validation.jsonl \
    --qa-test qa_dataset/test/test.jsonl


2025-12-31 13:15:58,085 - __main__ - INFO - Pipeline initialized
2025-12-31 13:15:58,085 - __main__ - INFO -   Output: results
2025-12-31 13:15:58,085 - __main__ - INFO -   Train: qa_dataset/train/train.jsonl
2025-12-31 13:15:58,085 - __main__ - INFO -   Test: qa_dataset/test/test.jsonl
2025-12-31 13:15:58,085 - __main__ - INFO -   Val: qa_dataset/validation/validation.jsonl
2025-12-31 13:15:58,085 - __main__ - INFO - ================================================================================
2025-12-31 13:15:58,085 - __main__ - INFO - BiEncoder Training & Evaluation Pipeline
2025-12-31 13:15:58,085 - __main__ - INFO - ================================================================================
2025-12-31 13:15:58,085 - __main__ - INFO - Output directory: results
2025-12-31 13:15:58,085 - __main__ - INFO - Models to train: ['bert-base-cased', 'scibert', 'pubmedbert', 'biobert', 'deberta-v3-base']
2025-12-31 13:15:58,095 - src.SR.BI.BiEncoder - INFO - ✓ MPS (Apple Silicon) avai

In [25]:
from pathlib import Path

from src.SR.BI.BiEncoderInference import BiEncoderInference

# Load a trained model
model = BiEncoderInference(
    model_dir=Path("results/models/bert-base-cased"),
    model_name="bert-base-cased"
)

# Set corpus
documents = [
    """
        Title: Global Prevalence of Hypertension: A Systematic Review

        Methods:
        We searched PubMed, Scopus, and Web of Science from January 1990 to March 2024. 
        After removing duplicates, 1,240 records were screened, and 137 full-text articles were assessed for eligibility.
        Seventy articles were excluded due to incomplete data, leaving 67 studies for analysis. 
        Two reviewers independently screened all titles and abstracts.

        Results:
        A total of 67 studies (n = 88,345 participants) from 25 countries were included. 
        The largest sample sizes were from China (n = 35,812), the United States (n = 10,437), and Germany (n = 7,670). 
        Other notable sample sizes included Bangladesh (n = 2,909), Ethiopia (n = 2,033), Egypt (n = 3,204), and Vietnam (n = 2,945).
        Fourteen were randomized controlled trials, ten cohort studies, and three case-control studies.

        Discussion:
        The global prevalence of hypertension remains high, particularly in low- and middle-income countries.

    """,
    "PubMedBERT is trained on biomedical text",
    "Antibiotic resistance is a global health threat",
]
model.set_corpus(documents)

# Single similarity
result = model.similarity(
    query="WHat is the total number of studies",
    context="Global Prevalence of Hypertension: A Systematic Review"
)
print(f"Score: {result.score:.4f}")
print(f"Model: {result.model}")

# Retrieve top-3 documents
results = model.retrieve(
    query="WHat is the total number of studies?",
    top_k=3
)
for r in results:
    print(f"[{r.rank}] ({r.model}) {r.context}: {r.score:.4f}")


Batches: 100%|██████████| 1/1 [00:00<00:00, 18.33it/s]

Score: 0.8665
Model: bert-base-cased
[1] (bert-base-cased) PubMedBERT is trained on biomedical text: 0.9108
[2] (bert-base-cased) 
        Title: Global Prevalence of Hypertension: A Systematic Review

        Methods:
        We searched PubMed, Scopus, and Web of Science from January 1990 to March 2024. 
        After removing duplicates, 1,240 records were screened, and 137 full-text articles were assessed for eligibility.
        Seventy articles were excluded due to incomplete data, leaving 67 studies for analysis. 
        Two reviewers independently screened all titles and abstracts.

        Results:
        A total of 67 studies (n = 88,345 participants) from 25 countries were included. 
        The largest sample sizes were from China (n = 35,812), the United States (n = 10,437), and Germany (n = 7,670). 
        Other notable sample sizes included Bangladesh (n = 2,909), Ethiopia (n = 2,033), Egypt (n = 3,204), and Vietnam (n = 2,945).
        Fourteen were randomized contro

In [1]:
import json
from pathlib import Path
from src.Commands.regexp import searchRegEx
from src.SR.paper_extractor import PaperExtractor

# 2. Sample Paper Text (Abstract)
sample_text = """
Background: We conducted a systematic review to assess the effectiveness of COVID-19 vaccines in adolescents.
Methods: We searched PubMed until January 15 2024. We included RCTs and observational studies.
Results: The study included 5000 participants. The effectiveness was 95% against infection. 
Mortality rates were low. No severe adverse events were reported in immunocompromised patients.
The study was conducted across Germany and USA.
"""
# Your FULL searchRegEx (500+ keywords) now works!
extractor = PaperExtractor(
    model_path=Path("results/models/bert-base-cased"),
    search_definitions=searchRegEx,  # Your complete dict
    threshold=0.65
)

# # Extract from paper
# data = extractor.extract(sample_text)

# # Get clean JSON
# print(json.dumps(data, indent=2))



/Users/awotoroebenezer/Desktop/SenseBackend/.venv_/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:src.SR.paper_extractor:Loading BiEncoder from results/models/bert-base-cased
INFO:src.SR.BI.BiEncoder:✓ MPS (Apple Silicon) available
INFO:src.SR.BI.BiEncoderInference:Loading model: bert-base-cased
INFO:src.SR.BI.BiEncoderInference:  Path: results/models/bert-base-cased
INFO:src.SR.BI.BiEncoderInference:  Device: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: results/models/bert-base-cased
INFO:src.SR.BI.BiEncoderInference:  Embedding dimension: 768
INFO:src.SR.paper_extractor:Building semantic tag index...
INFO:src.SR.paper_extractor:Encoding 272 keywords...
Batches: 100%|██████████| 9/9 [00:00<00:00, 15.05it/s]
INFO:src.SR.paper_extractor:✓ Built index with 

In [2]:
data = extractor.extract(sample_text)

INFO:src.SR.paper_extractor:Starting extraction...
INFO:src.SR.paper_extractor:Found 8 sentences
INFO:src.SR.paper_extractor:Encoding sentences...
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.43it/s]
INFO:src.SR.paper_extractor:Computing semantic similarities...
Batches: 100%|██████████| 1/1 [00:00<00:00, 21.25it/s]
INFO:src.SR.paper_extractor:Matching tags (threshold=0.65)...
INFO:src.SR.paper_extractor:Matched 272 tags
INFO:src.SR.paper_extractor:✓ Extraction complete


In [3]:
print(json.dumps(data, indent=2))

{
  "date of last literature search": "January 15 2024",
  "number of studies": "5000",
  "Population": {
    "age__group": [
      "nb",
      "chi",
      "ado",
      "adu",
      "eld"
    ],
    "specific__group": [
      "hcw",
      "pw",
      "tra",
      "cg"
    ],
    "immune__status": [
      "imu",
      "hty"
    ]
  },
  "topic": {
    "eff": [
      "eff"
    ],
    "safety": [
      "saf"
    ],
    "risk__factor": [
      "rf"
    ],
    "coverage": [
      "cov"
    ],
    "acceptance": [
      "kaa"
    ],
    "adm": [
      "adm"
    ],
    "eco": [
      "eco"
    ],
    "modeling": [
      "mod"
    ],
    "ethical__issues": [
      "eth"
    ]
  },
  "outcome": {
    "infection": [
      "inf"
    ],
    "hospital": [
      "hos"
    ],
    "icu": [
      "icu"
    ],
    "death": [
      "dea"
    ]
  },
  "intervention": {
    "vpd": [
      "covid",
      "aden",
      "anth",
      "camp",
      "chol",
      "coxi",
      "diph",
      "ebol",
      "ente"